In [ ]:
from lusidtools.jupyter_tools import toggle_code

"""Loading and calculating returns

Demonstration of how to load and calculate returns in LUSID.

Attributes
----------
Returns
"""

toggle_code("Toggle Docstring")

In [ ]:
# %load_ext lab_black
# %load_ext nb_black

# Loading and calculating returns in LUSID

This notebook shows how you can load and calculate returns in LUSID. In the cells below, we load a CSV file of daily returns over a 5 year period (approximately). We then use LUSID to calculate various return metrics:

* Daily
* Since inception
* Week-to-date (WTD), Month-to-date (MTD), Quarter-to-date (QTD), Year-to-date (YTD)
* 1 month (1M), 3 months (3M), 1 Year (1Y)
* Since a specific date
* Annualised since incorporation
* 5 year annualised


# Setup LUSID

In [ ]:
# Import general purpose packages
import os
import json
import pandas as pd
import numpy as np
import datetime
import pytz
import warnings

warnings.filterwarnings("ignore", module="matplotlib*")

import matplotlib.pyplot as plt

from datetime import datetime, timedelta

# Import lusid specific packages
import lusid
import lusid.models as models

from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken

# Set display configuration
pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", lambda x: "%.5f" % x)
pd.set_option("display.max_rows", 3500)
pd.set_option("max_colwidth", 20)

# Use line magic function to enable matplotlib to work interactively with iPython
%matplotlib inline

# Set style to fivethirtyeight to create clean and clear looking graphs
plt.style.use("fivethirtyeight")

# Define a dictionary containing default plotting configurations
params = {
    "legend.fontsize": "small",
    "figure.figsize": (12, 4.5),
    "axes.labelsize": "small",
    "axes.titlesize": "medium",
    "xtick.labelsize": "small",
    "ytick.labelsize": "small",
}

plt.rcParams.update(params)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

api_status = pd.DataFrame(
    api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().to_dict()
)

display(api_status)

# Setup defaults

In [ ]:
# Define a scope to hold data

scope = "returns"
portfolio_code = "UK_EQUITY"

# Define the APIs we use

transaction_portfolios_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
portfolios_api = api_factory.build(lusid.api.PortfoliosApi)

# Define dates
start_date = datetime(year=2020, month=1, day=2, tzinfo=pytz.UTC).isoformat
number_of_days = 366

# Create a portfolio

In [ ]:
portfolios = [portfolio_code]

for port in portfolios:

    try:

        response = transaction_portfolios_api.create_portfolio(
            scope=scope,
            create_transaction_portfolio_request=models.CreateTransactionPortfolioRequest(
                display_name=port, code=port, created="2013-12-30", base_currency="GBP"
            ),
        )
    except lusid.ApiException as e:
        print(json.loads(e.body)["title"])

# Upload a DataFrame of returns from CSV file to LUSID

In [ ]:
returns_df = pd.read_csv("data/performance_data.csv", dtype={"daily_returns": np.int64})
returns_df["date"] = pd.to_datetime(returns_df["date"], format="%d/%m/%Y")
returns_df = returns_df.rename(
    columns={"mv": "market_value", "returns": "daily_returns"}
)

returns_df.head()

> <b> NOTE: We have truncated the upsert to 10 rows. Remove the cell below if you want to load all 2500+ rows of daily returns.</b>

In [ ]:
returns_df = returns_df[:10]

In [ ]:
time_series_of_returns = [
    models.PerformanceReturn(
        effective_at=row["date"].isoformat()[:10],
        rate_of_return=row["daily_returns"],
        closing_market_value=row["market_value"],
        period="Daily",
    )
    for _, row in returns_df.iterrows()
]

return_code = "daily_return"

upsert_returns = portfolios_api.upsert_portfolio_returns(
    scope=scope,
    code=portfolio_code,
    return_code=return_code,
    return_scope=scope,
    performance_return=time_series_of_returns,
)

# Retrieve returns from LUSID

In [ ]:
from_effective_at = datetime(year=2013, month=12, day=31, tzinfo=pytz.utc).isoformat()
to_effective_at = datetime(year=2020, month=5, day=3, tzinfo=pytz.utc).isoformat()

returns = portfolios_api.get_portfolio_aggregated_returns(
    scope=scope,
    code=portfolio_code,
    aggregated_returns_request={
        "metrics": [
            {"window": "1D"},
            {"window": "INC"},
            {"window": "WTD"},
            {"window": "MTD"},
            {"window": "QTD"},
            {"window": "YTD"},
            {"window": "1M-ROLLING"},
            {"window": "3M-ROLLING"},
            {"window": "1Y-ROLLING"},
            {"window": "SINCE(2014-03-19)"},
            {"window": "INC", "annualised": "true", "alias": "ANNUALISED(INC)"},
            {
                "window": "5Y-ROLLING",
                "annualised": "true",
                "alias": "ANNUALISED(5Y-ROLLING)",
            },
        ],
        "returnIds": [{"code": return_code, "scope": scope}],
        "recipeId": {"code": "FxRates", "scope": scope},
        "period": "Daily",
        "outputFrequency": "Daily",
    },
    from_effective_at=from_effective_at,
    to_effective_at=to_effective_at,
)

In [ ]:
data = returns.results[f"{scope}/{return_code}"]

summary_df = pd.DataFrame(
    columns=[
        "date",
        "opening_mv",
        "closing_mv",
        "day",
        "inception",
        "mtd",
        "ytd",
        "qtd",
        "wtd",
        "1m-rolling",
        "3m-rolling",
        "1y-rolling",
        "since_20140319",
        "annualised_inc",
        "annualised_5y-rolling",
    ]
)


def make_row(item):
    row = {}
    row["date"] = item.effective_at.strftime(format="%Y-%m-%d")
    row["opening_mv"] = item.opening_market_value
    row["closing_mv"] = item.opening_market_value
    row["day"] = item.metrics_value.get("1D")
    row["inception"] = item.metrics_value.get("INC")
    row["mtd"] = item.metrics_value.get("MTD")
    row["ytd"] = item.metrics_value.get("YTD")
    row["qtd"] = item.metrics_value.get("QTD")
    row["wtd"] = item.metrics_value.get("WTD")
    row["1m"] = item.metrics_value.get("1M-ROLLING")
    row["3m"] = item.metrics_value.get("3M-ROLLING")
    row["1y"] = item.metrics_value.get("1Y-ROLLING")
    row["since_20140319"] = item.metrics_value.get("SINCE(2014-03-19)")
    row["annualised_inc"] = item.metrics_value.get("ANNUALISED(INC)")
    row["annualised_5y"] = item.metrics_value.get("ANNUALISED(5Y-ROLLING)")
    return row


summary_df = pd.concat(
    [summary_df, pd.DataFrame((make_row(item) for item in data))]
).reset_index(drop=True)

summary_df.tail(10)

In [ ]:
def time_series_performance():
    ts_performance = summary_df.plot(y=["inception"], figsize=(12, 9))
    ts_performance.set_title(
        f"Performance of {portfolio_code} since inception",
        fontsize="large",
    )
    ts_performance.set_ylabel("Performance in %", fontsize="large")
    ts_performance.set_xlabel("Days since inception", fontsize="large")
    ts_performance.legend(prop={"size": 12})


time_series_performance()

# Returns Dashboard

To view the returns loaded above, navigate to the following link:

In [ ]:
import os

api_url = os.getenv("FBN_LUSID_API_URL")
url = f"{api_url[:-3]}app/dashboard/returns?dateFrom=2013-12-31T00:00:00.000Z&dateTo=2022-11-29T23:59:59.999Z&frequency=Daily&compositeMethod=Asset&period=Daily&scope=returns&code=UK_EQUITY&entityType=Portfolio&returnsScope=returns&returnsCode=daily_return"

print(url)